In [14]:
import rooms.dataset
import render
import torch
import torch.nn as nn
import metrics
import train
import os

In [15]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
dataset_name = "prova"

D = rooms.dataset.dataLoader(dataset_name)

In [17]:
#training parameters

n_fibonacci = 128 #128
late_stage_model= "UniformResidual" #"UniformResidual"
toa_perturb = True #True
model_transmission = False #False

skip_train = False #False
continue_train = False #False

n_epochs = 1 #1000
batch_size = 4 #4
lr = 1e-2 #1e-2
pink_noise_supervision = False #True
pink_start_epoch = 500 #500
fs = 48000 #48000

load_dir= None
save_dir= '~/prova_training'

In [18]:
R = render.Renderer(n_surfaces=len(D.all_surfaces), n_fibonacci=n_fibonacci,
                        late_stage_model=late_stage_model,
                        toa_perturb = toa_perturb, model_transmission=model_transmission).to(device)

In [19]:
# Utilizza più GPU se disponibili
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    R = nn.DataParallel(R).module

solo per training le prossime

In [8]:
#directional case
loss_fcn = metrics.training_loss_for_learned_bp

for listener_position in D.RIRs:
    for response in listener_position:
        response['t_response'] = torch.Tensor(response['t_response'][:R.RIR_length])
        response['t_response'].to(device) 

gt_audio = D.RIRs
rendering_method = render.Renderer.render_RIR_learned_beampattern

In [8]:
def initialize(indices, source_xyz, listener_xyzs, surfaces, load_dir):
    Ls = []

    for idx in indices:
        L= render.get_listener(source_xyz=source_xyz, listener_xyz = listener_xyzs[idx], surfaces = surfaces, 
                               load_dir = load_dir, load_num = idx, speed_of_sound = D.speed_of_sound, 
                               max_order = D.max_order, parallel_surface_pairs = D.parallel_surface_pairs, 
                               max_axial_order = D.max_axial_order)
        Ls.append(L)
    return Ls

In [ ]:
"""
Training
"""
if not skip_train:
    print("Training")

    #Initialize Listeners
    Ls = initialize(indices=D.train_indices,
                    listener_xyzs=D.xyzs,
                    source_xyz=D.speaker_xyz,
                    surfaces=D.all_surfaces,
                    load_dir=load_dir)
            
    if continue_train:
        R.load_state_dict(torch.load(os.path.join(save_dir,"weights.pt"))['model_state_dict'])

    losses = train.train_loop(R=R, Ls=Ls, train_gt_audio=gt_audio[D.train_indices], D=D,
                        n_epochs = n_epochs, batch_size = batch_size, lr = lr, loss_fcn = loss_fcn,
                        save_dir=save_dir,
                        pink_noise_supervision = pink_noise_supervision,
                        pink_start_epoch=pink_start_epoch,
                        continue_train = continue_train, fs=fs)

else:
    R.load_state_dict(torch.load(os.path.join(save_dir,"weights.pt"))['model_state_dict'])
    R.train = False
    R.toa_perturb = False

prova di ascolto

In [20]:
import numpy as np
import evaluate

In [21]:
pt_file = torch.load(save_dir + '/weights.pt', map_location=device)
R.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
R.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

R.bp_ord_cut_freqs = nn.Parameter(pt_file['model_state_dict']['bp_ord_cut_freqs'])

In [66]:
listener = render.get_listener(source_xyz= np.array([5,8,1.5]), listener_xyz = np.array([5,9,1.5]), surfaces = D.all_surfaces, 
                               load_dir = load_dir, load_num = None, speed_of_sound = D.speed_of_sound, 
                               max_order = D.max_order, parallel_surface_pairs = D.parallel_surface_pairs, 
                               max_axial_order = D.max_axial_order)

Considered Paths:	6
Total Considered Paths, after Axial:	6
Valid Paths:	7


In [67]:
RIR = R.render_RIR(listener)

tensor([ 1.4154e-09,  2.0472e-09, -3.1614e-08,  ..., -1.1770e-04,
        -7.3632e-05, -1.1808e-04], device='cuda:0', grad_fn=<AddBackward0>)

In [68]:
predicted_music = evaluate.render_music(np.array([RIR.detach().cpu()]), np.array([D.music_dls[0]]), device = device)
predicted_music.shape

(1, 1, 480000)

In [69]:
D.music_dls.shape

(12, 1, 624000)

In [70]:
#import sounddevice as sd

#duration = predicted_music.shape[2]/fs  # Durata in secondi
#t = np.linspace(0, duration, int(fs * duration), endpoint=False)


#sd.play(predicted_music[0][0], samplerate=48000)
#sd.wait()  # Aspetta che la riproduzione termini


In [74]:
#sd.play(D.music_dls[9][0], samplerate=48000)
#sd.wait()  # Aspetta che la riproduzione termini

KeyboardInterrupt: 